In [26]:
import numpy as np
import scipy.special
from scipy.signal import convolve2d
import os
import os
import functools
import numpy as np
import scipy.special
from scipy.misc import imread
from skimage.color import rgb2gray
from scipy.signal import convolve2d
from scipy.ndimage import map_coordinates

In [27]:
def read_image(filename, representation):
    # filename - file to open as image
    # representation - is it a B&W or color image
    im = imread(filename)
    # check if it is a B&W image
    if(representation == 1):
        im = rgb2gray(im)
    # convert to float and normalize
    return (im / 255).astype(np.float32)

def relpath(filename):
    # converts relative paths to absolute
    # filename - relative path
    # returns - absolute path
    return os.path.join(os.path.dirname(__file__), filename)

def create_filter_vec(filter_size):
    # creates a binomial coefficient of length filter_size
    # filter_size - length of the coefficient array
    # returns - the binomial coefficient array

    # special case of an odd number.
    if filter_size == 1: return np.array([[0]])
    conv_ker =  np.array([[1, 1]])
    filter = conv_ker
    # using an O(logN) algorithm to compute the filter
    log2 = np.log2(filter_size - 1)
    whole = np.floor(log2).astype(np.int64)
    rest = (2**(log2) - 2**(whole)).astype(np.int64)
    for i in range(whole):
        filter = convolve2d(filter, filter).astype(np.float32)
    for i in range(rest):
        filter = convolve2d(filter, conv_ker).astype(np.float32)
    # normalize
    return (filter / np.sum(filter)).astype(np.float32)

def build_gaussian_pyramid(im, max_levels, filter_size):
    # calc the filter array
    filter_vec = create_filter_vec(filter_size)
    # create the entire array for better complexity
    pyr = [0] * (np.min([max_levels, np.log2(im.shape[0]).astype(np.int64) - 3,
                         np.log2(im.shape[1]).astype(np.int64) - 3]))
    pyr[0] = im
    # for each iter, use the last iter to calc the current iter. note i transpose twice. once to calc
    # the y conv and the second to flip back the image
    for i in range(1, len(pyr)):
        pyr[i] = scipy.ndimage.filters.convolve(pyr[i - 1], filter_vec, output = None, mode = 'mirror')
        pyr[i] = scipy.ndimage.filters.convolve(pyr[i].transpose(), filter_vec, output = None, mode = 'mirror')
        pyr[i] = (pyr[i].transpose()[::2, ::2]).astype(np.float32)
    return pyr, filter_vec

In [28]:
im1 = read_image("/home/itamar/Documents/ip/ex4/files/external/oxford1.jpg", 1)

pyr1, _ = build_gaussian_pyramid(im1, max_levels=3, filter_size=3)

In [41]:
bb = np.array([[0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5 ,6.5
          ,0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5 ,6.5
          ,0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5 ,6.5
          ,0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5 ,6.5
          ,0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5 ,6.5
          ,0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5 ,6.5
          ,0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5 ,6.5],
          [160.75, 160.75, 160.75, 160.75, 160.75, 160.75, 160.75
          ,161.75, 161.75, 161.75, 161.75, 161.75, 161.75, 161.75
          ,162.75, 162.75, 162.75, 162.75, 162.75, 162.75, 162.75
          ,163.75, 163.75, 163.75, 163.75, 163.75, 163.75, 163.75
          ,164.75, 164.75, 164.75, 164.75, 164.75, 164.75, 164.75
          ,165.75, 165.75, 165.75, 165.75, 165.75, 165.75, 165.75
          ,166.75, 166.75, 166.75, 166.75, 166.75, 166.75, 166.75]])
K = 7

In [54]:
bb.shape
a = bb[0]
b = bb[1]
c = np.array([b,a])
import math

In [55]:
map_coordinates(pyr1[2], c, order=1, prefilter=False).reshape(K, K)

array([[  4.44253528e-04,   1.41362092e-04,   7.68693571e-05,
          1.23648919e-04,   5.00089955e-04,   9.41918639e-04,
          1.06451684e-03],
       [  4.17557370e-04,   1.33181762e-04,   7.78376998e-05,
          1.33783426e-04,   5.11539867e-04,   9.46037530e-04,
          1.07191980e-03],
       [  3.98587028e-04,   1.28587883e-04,   8.15829480e-05,
          1.49279018e-04,   5.35091094e-04,   9.58444551e-04,
          1.07379677e-03],
       [  3.69321613e-04,   1.22075886e-04,   8.08141995e-05,
          1.58561204e-04,   5.54377912e-04,   9.61689744e-04,
          1.05731352e-03],
       [  3.33312753e-04,   1.12575421e-04,   7.91027414e-05,
          1.75967682e-04,   5.74724341e-04,   9.13641183e-04,
          9.51379538e-04],
       [  2.97695922e-04,   1.00632475e-04,   7.71745399e-05,
          1.89796323e-04,   5.92204684e-04,   9.42805025e-04,
          1.03746413e-03],
       [  2.67473632e-04,   9.90625995e-05,   8.08675977e-05,
          2.04091717e-04,   6.48

In [58]:
b = np.array([[1,2,3], [4,5,6], [7,8,9]])
b[1,0]

4

In [59]:
np.dstack(np.meshgrid(np.arange(5), np.arange(5))).reshape(-1, 2)

array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [0, 1],
       [1, 1],
       [2, 1],
       [3, 1],
       [4, 1],
       [0, 2],
       [1, 2],
       [2, 2],
       [3, 2],
       [4, 2],
       [0, 3],
       [1, 3],
       [2, 3],
       [3, 3],
       [4, 3],
       [0, 4],
       [1, 4],
       [2, 4],
       [3, 4],
       [4, 4]])